In [1]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_TOKEN")
login(token=hf_token)

In [2]:
from perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv() 

client = Perplexity() # PERPLEXITY_API_KEY is read from environment variable

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

Introducing Comet: Browse at the speed of thought: https://www.perplexity.ai/hub/blog/introducing-comet
Perplexity: https://www.perplexity.ai
Perplexity Blog: https://www.perplexity.ai/hub
Browse at the speed of thought: https://www.perplexity.ai/comet/
Perplexity AI: https://en.wikipedia.org/wiki/Perplexity_AI
Changelog: https://docs.perplexity.ai/changelog/changelog
Comet Browser Review 2025: Features, Pricing, Pros & Cons: https://efficient.app/apps/comet
Getting Started with Perplexity: https://www.perplexity.ai/hub/getting-started
perplexity-ai-toolkit/.github/CHANGELOG.md at main · RMNCLDYO/perplexity-ai-toolkit: https://github.com/RMNCLDYO/perplexity-ai-toolkit/blob/main/.github/CHANGELOG.md
Perplexity's AI-Native Browser Comet is HERE: https://www.youtube.com/watch?v=ri_bFrDp44M


In [3]:
import json
from pprint import pprint

goal = "Analyze APPL stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create a detailed plan to achieve this goal:
    Goal: {goal}
    
    Available tools: web search, financial data API, news aggregator, sentiment analysis tool

    Create a plan with:
    1. Logical research steps (3-7 steps)
    2. Estimated time in minutes
    3. Required tools for each step
    4. Success criteria to evaluate completion
    
    Answer in JSON format:
    {{
        "goal": "...",
        "steps": ["step1", "step2", ...],
        "estimated_time": 30,
        "required_tools": ["tool1", "tool2", ...],
        "success_criteria": ["criteria1", "criteria2", ...]
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)


### Presenting the data generated

In [4]:

#print("Raw Response:\n", response.choices[0].message.content)
plan_data = json.loads(response.choices[0].message.content)

pprint(plan_data)

{'estimated_time': 45,
 'goal': 'Analyze AAPL stock for a potential investment',
 'required_tools': ['financial data API',
                    'web search',
                    'news aggregator',
                    'sentiment analysis tool'],
 'steps': ['Collect recent and historical financial data for Apple Inc. '
           'including income statement, balance sheet, cash flow statements, '
           'and key financial ratios.',
           'Perform a fundamental analysis using financial ratios '
           '(profitability, liquidity, solvency, operating efficiency) and '
           'valuation models such as discounted cash flow (DCF) and relative '
           'valuation against peers.',
           'Gather and analyze recent news articles and earnings reports using '
           'a news aggregator to assess qualitative factors impacting Apple.',
           'Use a sentiment analysis tool on the news and social media '
           'mentions of Apple to gauge market sentiment and identif

### Sentiment analysis using perplexity API

#### Get information from FinTools

In [6]:
from src.aggregator import FinancialAggregator, NewsAggregator


ticker = "PLTR"  # Example stock ticker
stock_info = FinancialAggregator().execute(ticker)
pprint(stock_info)

news_articles = NewsAggregator().execute(ticker, 5)
for article in news_articles:
    pprint(article)


{'52_week_high': 190.0,
 '52_week_low': 36.05,
 'avg_volume_30d': 57327227.0,
 'beta': 2.592,
 'company_summary': 'Palantir Technologies Inc. builds and deploys software '
                    'platforms for the intelligence community to assist in '
                    'counterterrorism investigations and operations in the '
                    'United States, the United Kingdom, and internationally. '
                    'It provides Palantir Gotham, a software platform, which '
                    'enables users to identify patterns hidden deep within '
                    'datasets, ranging from signals intelligence sources to '
                    'reports from confidential informants, as well as '
                    'facilitates the hand-off between analysts and operational '
                    'users, he',
 'current_price': 180.9499969482422,
 'dividend_yield': None,
 'historical_data': [{'Close': 157.08999633789062,
                      'Dividends': 0.0,
                      

#### Generate sentiment analysis and confidence score

In [7]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 {
  "stock": "PLTR",
  "sentiment": "Positive",
  "summary": "Palantir Technologies (PLTR) currently exhibits a positive sentiment driven by strong recent price performance, a substantial year-to-date gain of over 130%, and bullish technical patterns suggesting potential for continued upside. The company is recognized as a market leader in decision intelligence and AI/ML platforms with robust revenue growth (48% YoY), expanding customer base, and increasing spend per customer. Analysts have upgraded earnings estimates and recognize Palantir's unique moat and AI platform momentum despite a high valuation several times above the software sector average. The market expects continued growth with projections of 30% price appreciation over the next three months. However, some caution is warranted due to the stock's high P/E ratio and potential volatility. On balance, fundamentals and technicals support a positive outlook.",
  "confidence": 0.9
}
